In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import os
import re
import cv2
import glob
import time
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

import dask as dd
import dask.array as da
from dask.distributed import Client, progress

seed = 111
np.random.seed(seed)
%config IPCompleter.use_jedi = False

# Load the datasets

In [ ]:
data_path = Path("../input/rsna-miccai-brain-tumor-radiogenomic-classification/")
train_data = data_path / "train"
test_data = data_path / "test"

In [ ]:
train_df = pd.read_csv(data_path / "train_labels.csv")
print("Total number of training samples:", len(train_df))
train_df.head()

In [ ]:
# Because each directory in the training images have 5 digits in its name
# we will make a small change in our df
train_df["BraTS21ID"] = train_df["BraTS21ID"].apply(lambda x: str(x).zfill(5))
train_df.head()

**Visualizations**

In [ ]:
def dicom2array(path, voi_lut=True, fix_monochrome=True):
    # Original from: https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
    dicom = pydicom.read_file(path)
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    return data
random_idx = np.random.randint(len(train_df))
case = train_df["BraTS21ID"][random_idx]
label = train_df["MGMT_value"][random_idx]
scans = ["FLAIR", "T1w", "T1wCE", "T2w"]

all_slices = list(map(str, list((train_data / case / scans[0]).glob("*.dcm"))))

# Sort the slices
all_slices.sort(key=lambda x: int(re.sub('\D', '', x)))

nrows = len(all_slices) // 5
ncols = 5
f, ax = plt.subplots(nrows, ncols, figsize=(20, 40))

for i in range(len(all_slices)):
    img = dicom2array(all_slices[i])
    ax[i // ncols, i % ncols].imshow(img, cmap="gray")
    ax[i // ncols, i % ncols].axis('off')
plt.show()

# Process DICOM in parallel

In [ ]:
@dd.delayed
def read_dicom_file(filepath):
    return pydicom.read_file(filepath)

@dd.delayed
def apply_sequence(dicom):
    return apply_voi_lut(dicom.pixel_array, dicom)

@dd.delayed
def fix_monochrome(data):
    data = np.amax(data) - data
    return data

@dd.delayed
def preprocess_data(data):
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    return data
def process_dicom(slice_path):
    dicom = read_dicom_file(slice_path)
    data = apply_sequence(dicom)
    data = fix_monochrome(data)
    data = preprocess_data(data)
    return data

In [ ]:
client = Client()
client

In [ ]:
processed_slice_data = []
scans = ["FLAIR", "T1w", "T1wCE", "T2w"]
count = 0

start_time = time.time()

# I will process only first 10 samples to keep it simple!
for i in range(len(train_df.iloc[:10, :])):
    case = train_df["BraTS21ID"][i]
    outputs = []
    
    for scan in scans:
        all_slices = list((train_data / case / scan).glob("*.dcm"))
        all_slices.sort(key=lambda x: int(re.sub('\D', '', str(x))))
        outputs.append(process_dicom(slice_path) for slice_path in all_slices)
    
    processed_slice_data.append(dd.compute(outputs)[0])
    
    
    for data in processed_slice_data[i:]:
        count += len(data[0]) + len(data[1]) + len(data[2]) + len(data[3])
    print("Number of DICOM files processed: ", count)
    
    
print(f"\nTime taken to read all slices for 10 samples: {time.time()-start_time:.2f}")

In [ ]:
client.close()